# Fase 2: Limpieza y Transformación

#### 1. Importamos las librerías necesarias y agregamos la ruta de nuestro archivo .py

In [ ]:
import sys
import pandas as pd

# Agregamos la ruta
sys.path.append('../')

# Importamos la función procesar_dataframe de limpieza.py
from src.limpieza import procesar_dataframe

# Importamos el diccionario de variables.py
from src.variables import cambiar_tipo_columnas

#### 2. Cargamos el dataframe transformado, hacemos llamada a la función y comprobamos los cambios

In [ ]:
# Cargamos el DataFrame
df_csv = pd.read_csv('../data/transform_data/transform_Data.csv', low_memory=False)

# Llamamos a la función procesar_dataframe con los diccionarios importados
df = procesar_dataframe(df_csv, cambiar_tipo_columnas)

# Verificamos el resultado mostrando las primeras filas
df.dtypes

#### 3. Analizamos si existen o no valores duplicados, para poder reducir el tamaño del archivo y mejorar la velocidad del fichero

In [ ]:
# Comprobamos si existen valores duplicados
if df.duplicated().values.any():
    print("Existen valores duplicados por eliminar")
else:
    print("No existen valores duplicados")

# Comprobamos y verificamos que no existen valores duplicados
df.duplicated().sum()

#### 4. Analizamos si existen o no valores nulos

In [ ]:
# Calculamos cuantos valores nulos existen
print("Los valores nulos existentes por columna son:")
df.isnull().sum()

In [ ]:
# Calculamos el % de nulos en cada columna, para saber la cantidad de nulos que hemos obtenido
df.isnull().mean() * 100

#### 5. Conclusiones del cálculo de % de valores nulos

**Calculamos el % de valores nulos en cada columna, y obtenemos las siguientes conclusiones:**

1. Si una columna contiene el 100% de valores nulos, lo más eficiente es eliminar dicha columna (total_market_value, coach_name)<br>

2. Si menos del 5% de los valores de la columna son nulos, podemos sustituirlos por la mediana en columnas numéricas y la moda en columnas categóricas.<br>

3. Sustituir por la moda es adecuado para las columnas categóricas, ya que la moda es el valor más frecuente.<br>

4. Sustituir por la mediana es útil cuando la distribución de los datos es asimétrica (sesgada), ya que la mediana no se ve afectada por los valores extremos.<br>

In [ ]:
# Según el análisis eliminamos la columna Date, puesto que tiene el 100% de valores nulos
df = df.drop(['total_market_value', 'coach_name'], axis=1)

In [ ]:
# Sustituimos los nulos en las columnas numéricas, por la mediana de cada columna
numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

In [ ]:
# Sustituimos los nulos en las columnas categóricas, por la moda de cada columna
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])

In [ ]:
# Verificamos que ya no hay valores nulos
df.isnull().sum()

In [ ]:
# Verificamos que se aplicaron los cambios
df.head()

#### 6. Analizamos las columnas categóricas

In [ ]:
# Verificamos posibles erratas o valores mal escritos, con head mostramos los 20 valores de mayor frecuencia
for col in df.select_dtypes(include=['object']).columns:
    print(f"\n🔹 Columna: {col}")
    print(df_csv[col].value_counts(dropna=False).head(20))

In [ ]:
# Eliminamos espacios en blanco, espacios dobles o caracteres
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace(r'\s+', ' ', regex=True)

#### 7. Analizamos las columnas numéricas

In [ ]:
# Revisamos las estadísticas generales
df.select_dtypes(include=["number"]).describe()

In [ ]:
# Detectamos valores fuera de rango, por ejemplo si obtenemos una edad negativa
for col in df.select_dtypes(include=['number']).columns:
    print(f"\n🔹 Columna: {col}")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Máximo: {df[col].max()}, Mínimo: {df[col].min()}")

#### 8. Cálculo del Método de Rango Intercuartílico (IQR)

In [ ]:
# Detectamos valores bajos o altos que podrían causar conflicto
import numpy as np

for col in df.select_dtypes(include=['number']).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    print(f"\n🔹 Columna: {col} - Outliers encontrados: {outliers.shape[0]}")

#### 9. Guardamos el CSV con la limpieza final realizada

In [ ]:
df.to_csv('../data/transform_data/data_clean.csv', index=False)